In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

/Users/pdh21/anaconda3/envs/xidplus/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_GAMA-09/data/holes_GAMA-09_ukidss_k_O16_20180417.fits')


## Read in CIGALE predictions catalogue

In [5]:
cigale=Table.read('../../dmu28/dmu28_GAMA-09/data/GAMA-09_Ldust_prediction_results.fits')


In [6]:
cigale['id'].name = 'help_id'

In [7]:
cigale

help_id,bayes.dust.luminosity,bayes.dust.luminosity_err,best.omegacam_u,best.megacam_u,best.omegacam_g,best.suprime_g,best.decam_g,best.megacam_g,best.gpc1_g,best.suprime_r,best.gpc1_r,best.megacam_r,best.omegacam_r,best.decam_r,best.gpc1_i,best.megacam_i,best.omegacam_i,best.suprime_i,best.gpc1_z,best.vista_z,best.megacam_z,best.suprime_z,best.decam_z,best.gpc1_y,best.suprime_y,best.vista_y,best.ukidss_y,best.ukidss_j,best.vista_j,best.ukidss_h,best.vista_h,best.vista_ks,best.ukidss_k
,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy
bytes27,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J082912.113+014254.016,1.7756811583627856e+39,1.8560994213383705e+39,0.0006849655648856183,0.0008213525215570016,0.0014418320784829225,0.0014851635254180297,0.0014918097398899804,0.0015248420785701489,0.0015223639970083685,0.0029236244063870925,0.0029323451806340392,0.0029942044550754767,0.003071945806420398,0.003209946563396643,0.00570727344151689,0.005874273794688017,0.006124423478050316,0.006517263751715815,0.012043668064701779,0.012387236998768944,0.012730939515909855,0.012885496989621615,0.013894760724993507,0.015677503791890205,0.015995150517214757,0.01681551604212792,0.017040868125072547,0.024214296725186508,0.02429380670766754,0.03624442746030166,0.03653520866519151,0.053322189508958964,0.05449461857253117
HELP_J082913.120+014309.267,2.7021265703371776e+37,2.4317979538966422e+36,0.05731104329672883,0.056223578961906474,0.06033796980008266,0.06082318037150582,0.06095714255843882,0.060962559918838935,0.06123168837954967,0.05832850068459655,0.05801617455383524,0.0574112860668829,0.05741336122906761,0.056988475834606624,0.05282675100407725,0.053128606958725344,0.05306034143787001,0.05267269867326263,0.051136801552879335,0.051096163964210975,0.051204577645507635,0.05120398359763581,0.05122226385822606,0.05100542972748624,0.05089417995814178,0.051669357982216245,0.052035689812129096,0.053585319826126916,0.05354605081471752,0.05234953875464276,0.052339845325697625,0.052655239140812916,0.0514389634370146
HELP_J082913.510+014205.882,4.570194861817776e+37,2.8856447896428874e+37,0.001562775024576193,0.0019241849445430818,0.004073465839980223,0.004389458937042463,0.004445338813423105,0.0046145338160607985,0.004617749979019916,0.013338348306415565,0.013418082449615516,0.013756858071279767,0.014021819449388108,0.01455585883157852,0.021016747199825565,0.02094968585405312,0.02126166115867816,0.021866551844849095,0.027435222606925637,0.02793834579131693,0.02841994636277121,0.02873496116158678,0.030069647897269628,0.032687682671318524,0.03361467379577717,0.03654262073705172,0.03715744099421227,0.05233128843384468,0.0525172784307798,0.06935969566164578,0.06970494617425149,0.09128148854208384,0.09383798173773936
HELP_J082913.660+014249.800,1.5495572578977246e+39,2.0426412306543896e+39,0.0007113458085920682,0.0008620998241307844,0.0014636804186210113,0.0015183653964489588,0.001528907940829464,0.0015650614718851665,0.001565504843687772,0.003486666574064081,0.0034925073419556116,0.003569124910119692,0.003707632994832172,0.003974755257606487,0.009555373494452808,0.009514978147316,0.009834455087319656,0.01053562651452195,0.017090714828527045,0.01772392654238113,0.017995607058220953,0.018406453207120137,0.01938260611424127,0.021299714542906932,0.022002392212932025,0.024543109914095355,0.025282734427615784,0.03732825099814391,0.037421056282661354,0.061449745967043015,0.06211407956664785,0.08968285075940517,0.09202998934088379
HELP_J082914.042+014215.484,1.9836831174912827e+39,1.1247053696095027e+38,6.298024535939316,6.138427217228513,6.478874044154541,6.499129387376567,6.5054687822726445,6.517273282028814,6.524689489260027,5.877294815665693,5.8475216229471565,5.792011

## Read in photoz

In [8]:
photoz=Table.read('../../dmu24/dmu24_GAMA-09/data/master_catalogue_gama-09_20171206_photoz_20180213_r_optimised.fits')

In [9]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
bytes27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6
HELP_J084930.013-020417.068,132.37505356730503,-2.07140772342553,30302,0.821,0.3029,1.3791,0.798,-99.0,-99.0,-99.0,-99.0,0.7474615125223136,-99.0,-99.0,-99.0,-99.0,
HELP_J085343.238-011847.076,133.430156467305,-1.3130766064255301,30303,1.1554,0.3875,1.9962,0.797,-99.0,-99.0,-99.0,-99.0,1.017680534872741,-99.0,-99.0,-99.0,-99.0,
HELP_J084950.135-013553.799,132.45889656730503,-1.5982775844255301,30304,1.0387,0.3587,1.78,0.8,-99.0,-99.0,-99.0,-99.0,0.9290274556241438,-99.0,-99.0,-99.0,-99.0,
HELP_J090051.096-013904.046,135.21289956730502,-1.65112388142553,30306,1.0914,0.3751,1.8732,0.796,-99.0,-99.0,-99.0,-99.0,0.9699034467909559,-99.0,-99.0,-99.0,-99.0,
HELP_J085606.834-010133.117,134.02847336730503,-1.02586584842553,30307,1.0371,0.3628,1.7717,0.796,-99.0,-99.0,-99.0,-99.0,0.9232576825764145,-99.0,-99.0,-99.0,-99.0,
HELP_J090529.562-011041.306,136.373173467305,-1.17814049642553,30308,1.1091,0.3792,1.9078,0.797,-99.0,-99.0,-99.0,-99.0,0.9817405966027227,-99.0,-99.0,-99.0,-99.0,
HELP_J090046.918-011856.173,135.19549056730503,-1.3156034864255302,30312,0.9841,0.3466,1.6739,0.796,-99.0,-99.0,-99.0,-99.0,0.8833496027298801,-99.0,-99.0,-99.0,-99.0,
HELP_J085032.410-011540.279,132.63504356730502,-1.26118848342553,30313,1.109,0.3792,1.9078,0.797,-99.0,-99.0,-99.0,-99.0,0.9817405966027227,-99.0,-99.0,-99.0,-99.0,
HELP_J085443.327-011651.769,133.68052766730503,-1.28104686042553,30314,0.9931,0.3506,1.6899,0.796,-99.0,-99.0,-99.0,-99.0,0.8889996515380697,-99.0,-99.0,-99.0,-99.0,


## Join CIGALE and photoz tables

In [11]:
prior=join(cigale,photoz,keys='help_id')

## Import model/s to find f250_pred


In [17]:
# We have fit a model to the data on other fields to obtained an empirical relationship
# between the ratio (Ldust/f250) and redshift

import pickle

# Function used to fit the data (exponential)
def func(p, x):
    p1, p2 = p
    return p1*np.log(x)+p2

# read model (fitted to all data - XID+)
c_model1a = pickle.load(open('../Ldust_investigation/cosmos_model_usingChi2.pkl', 'rb'))

## Find Ldust/f250 ratio relationship for each model

In [18]:
# ## NEW MODELS - XID+ ##
#######################

# Read model and apply ratio - z function to the dataset
logLflux_ratioC = func(c_model1a.beta, prior['z1_median'])
# Extract the flux prediction from the ratio (Ldust/f250) relationship
Lflux_ratioC = 10**logLflux_ratioC
f_pred = prior['bayes.dust.luminosity'] / Lflux_ratioC


In [19]:
# Different flux cuts
# After an analysis running XID+ on a few tiles with different flux cuts, 
# we decided to use f_pred > 5mJy as our flux cut
print('len total prior list: ', len(prior))
print('len C1 cut > 0.2: ', len(prior[f_pred > 0.2]))
print('len C1 cut > 1: ', len(prior[f_pred > 1]))
print('len C1 cut > 2: ', len(prior[f_pred > 2]))
print('len C1 cut > 3: ', len(prior[f_pred > 3]))
print('len C1 cut > 4: ', len(prior[f_pred > 4]))
print('len C1 cut > 5: ', len(prior[f_pred > 5]))
print('len C1 cut > 7: ', len(prior[f_pred > 7]))
print('len C1 cut > 10: ', len(prior[f_pred > 10]))
print('len C1 cut > 20: ', len(prior[f_pred > 20]))
print('len C1 cut > 50: ', len(prior[f_pred > 50]))
print('len C1 cut > 100: ', len(prior[f_pred > 100]))

len total prior list:  1815905
len C1 cut > 0.2:  1815839
len C1 cut > 1:  1815271
len C1 cut > 2:  1813150
len C1 cut > 3:  1808866
len C1 cut > 4:  1801662
len C1 cut > 5:  1789200
len C1 cut > 7:  1744055
len C1 cut > 10:  1644324
len C1 cut > 20:  1310616
len C1 cut > 50:  725157
len C1 cut > 100:  359711


In [20]:
prior_sel = prior[['help_id', 'RA', 'DEC']]
prior_sel['f250_pred'] = f_pred

In [21]:
prior_sel['DEC'].name='Dec'

In [33]:
prior_sel=prior_sel[f_pred>5]

In [34]:
prior_sel

help_id,RA,Dec,f250_pred
bytes27,float64,float64,float64
HELP_J082912.113+014254.016,127.30046915959387,1.7150045197524941,98.18765923094588
HELP_J082913.120+014309.267,127.30466646191715,1.7192408866967637,54.04979731148658
HELP_J082913.510+014205.882,127.30629302011518,1.7016338651294207,25.480352990433815
HELP_J082913.660+014249.800,127.30691774978729,1.7138334243403401,125.67785497969088
HELP_J082914.042+014215.484,127.30850860551412,1.704301179877907,5770.518547862265
HELP_J082914.054+014304.068,127.3085598984684,1.717796579285437,51.46665197743187
HELP_J082914.356+014322.049,127.30981575517112,1.722791294977396,19.926297833340957
HELP_J082914.699+014151.046,127.31124445581914,1.6975128960570705,17.95066408832632
HELP_J082915.204+014144.875,127.31335155513106,1.695798615975325,18880.984024807076


## Read in Maps

In [25]:

pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-09_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-09_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-09_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [26]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [27]:
## Set XID+ prior class

In [36]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA09_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA09_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA09_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )
prior500.prior_bkg(-5.0,5)

In [37]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [38]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 4959 tiles required for input catalogue and 102 large tiles
writing total_bytes=1304735243...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1304735243)... done.


SystemExit: 

In [ ]:
prior250.nsrc